### Get sample data

In [45]:
import csv

In [47]:
datas = []
with open('sample_data.csv', mode='r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        row = dict(row)
        row["item_id"] = int(row["item_id"])
        row["item_price"] = float(row["item_price"])
        row["rating"] = float(row["rating"])
        datas.append(dict(row))

In [48]:
datas

[{'item_id': 1,
  'item_name': 'TNG 3 ply washable cloth mask Replace Surgical face masks Antibacterial water resistant',
  'item_price': 8.5,
  'category': 'Protective Clothing and Equipment',
  'rating': 4.8},
 {'item_id': 2,
  'item_name': 'HUAWEI MateBook D15 (15.6" FHD IPS Laptop / AMD Ryzen 5 3500U + Radeon Vega 8 / 8GB RAM)',
  'item_price': 798.0,
  'category': 'Computers & Peripherals',
  'rating': 4.9},
 {'item_id': 3,
  'item_name': 'Mohamm English Letter & Number Sticker',
  'item_price': 0.27,
  'category': 'Stationery & Supplies ',
  'rating': 4.9},
 {'item_id': 4,
  'item_name': 'Super Hero Canvas Zipper Pencil Case Fashion Student Stationery Supplies',
  'item_price': 0.6,
  'category': 'Stationery & Supplies ',
  'rating': 4.8},
 {'item_id': 5,
  'item_name': '[WFRV]Baby Girl Birthday Party Dress Costume Princess Belle Kids Clothes ',
  'item_price': 6.3,
  'category': 'Kids Fashion ',
  'rating': 4.9}]

### Create index in elastic search

In [134]:
from elasticsearch import Elasticsearch
from pprint import pprint

In [140]:
client = Elasticsearch('localhost')

In [141]:
client.ping()

True

In [143]:
mappings = {
    'mappings': {
        'properties': {
            'category': {
                'fields': {
                    'keyword': {
                        'ignore_above': 256, 'type': 'keyword'
                    }
                },
                'type': 'text'
            },
            'item_id': {
                'type': 'long'
            },
            'item_name': {
                'fields': {
                    'keyword': {
                        'ignore_above': 256, 'type': 'keyword'
                    }
                },
                'type': 'text'
            },
            'item_price': {
                'type': 'float'
            },
            'rating': {
                'type': 'float'
            }
        }
    }
}

In [144]:
# !curl -X DELETE "localhost:9200/shopee?pretty"

In [145]:
client.indices.create(index="shopee", body=mappings)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'shopee'}

### Insert data into elastic search

In [146]:
for index, data in enumerate(datas, 1):
    result = client.index(index="shopee", id=index, body=data)
    print(f"Result : {result}")

Result : {'_index': 'shopee', '_type': '_doc', '_id': '1', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}
Result : {'_index': 'shopee', '_type': '_doc', '_id': '2', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}
Result : {'_index': 'shopee', '_type': '_doc', '_id': '3', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1}
Result : {'_index': 'shopee', '_type': '_doc', '_id': '4', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1}
Result : {'_index': 'shopee', '_type': '_doc', '_id': '5', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 4, '_primary_term': 1}


### Basic query

In [147]:
query = {
    "query": {
        "match_all": {}
    }
}

In [148]:
client.search(index="shopee", body=query)

{'took': 288,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 5, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'shopee',
    '_type': '_doc',
    '_id': '1',
    '_score': 1.0,
    '_source': {'item_id': 1,
     'item_name': 'TNG 3 ply washable cloth mask Replace Surgical face masks Antibacterial water resistant',
     'item_price': 8.5,
     'category': 'Protective Clothing and Equipment',
     'rating': 4.8}},
   {'_index': 'shopee',
    '_type': '_doc',
    '_id': '2',
    '_score': 1.0,
    '_source': {'item_id': 2,
     'item_name': 'HUAWEI MateBook D15 (15.6" FHD IPS Laptop / AMD Ryzen 5 3500U + Radeon Vega 8 / 8GB RAM)',
     'item_price': 798.0,
     'category': 'Computers & Peripherals',
     'rating': 4.9}},
   {'_index': 'shopee',
    '_type': '_doc',
    '_id': '3',
    '_score': 1.0,
    '_source': {'item_id': 3,
     'item_name': 'Mohamm English Letter & Number Sticker',
     'item_pric

### Query using term query

In [153]:
query = {
    "query": {
        "term": {
            "category": "Stationery"
        }
    }
}

In [154]:
client.search(index="shopee", body=query)

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

### Query using match query (Better search)

In [170]:
query = {
    "query": {
        "match": {
            "item_name": "huawei"
        }
    }
}

In [171]:
client.search(index="shopee", body=query)

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.1689311,
  'hits': [{'_index': 'shopee',
    '_type': '_doc',
    '_id': '2',
    '_score': 1.1689311,
    '_source': {'item_id': 2,
     'item_name': 'HUAWEI MateBook D15 (15.6" FHD IPS Laptop / AMD Ryzen 5 3500U + Radeon Vega 8 / 8GB RAM)',
     'item_price': 798.0,
     'category': 'Computers & Peripherals',
     'rating': 4.9}}]}}

### Query using match phrase query (Better search)

In [196]:
query = {
    "query": {
        "match_phrase": {
            "item_name": {
                "query": "huawei laptop",
                "slop": 6
            }
        }
    }
}

In [197]:
client.search(index="shopee", body=query)

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.5724969,
  'hits': [{'_index': 'shopee',
    '_type': '_doc',
    '_id': '2',
    '_score': 0.5724969,
    '_source': {'item_id': 2,
     'item_name': 'HUAWEI MateBook D15 (15.6" FHD IPS Laptop / AMD Ryzen 5 3500U + Radeon Vega 8 / 8GB RAM)',
     'item_price': 798.0,
     'category': 'Computers & Peripherals',
     'rating': 4.9}}]}}

### Query using boolean statements (should, must)

In [216]:
query = {
    "query": {
        "bool": {
            "must": [ # And condition
                    {
                        "match": {
                            "item_name": "huawei"
                        }
                    }      
            ],
            "should": [ # Or condition
                {
                    "match": {
                        "item_name": "huawei"
                    },
                    "match": {
                        "item_name": "Birthday party"
                    },
                }
            ]
        }
    }
}

In [218]:
client.search(index="shopee", body=query)

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.1689311,
  'hits': [{'_index': 'shopee',
    '_type': '_doc',
    '_id': '2',
    '_score': 1.1689311,
    '_source': {'item_id': 2,
     'item_name': 'HUAWEI MateBook D15 (15.6" FHD IPS Laptop / AMD Ryzen 5 3500U + Radeon Vega 8 / 8GB RAM)',
     'item_price': 798.0,
     'category': 'Computers & Peripherals',
     'rating': 4.9}}]}}

### Aggregation

In [234]:
query = {
    "query": {
        "match_all": {}
    },
    "aggs": {
        "by_rating": {
            "histogram": {
                "field": "rating",
                "interval": 0.1
            }
        }
    }
}

In [235]:
client.search(index="shopee", body=query)

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 5, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'shopee',
    '_type': '_doc',
    '_id': '1',
    '_score': 1.0,
    '_source': {'item_id': 1,
     'item_name': 'TNG 3 ply washable cloth mask Replace Surgical face masks Antibacterial water resistant',
     'item_price': 8.5,
     'category': 'Protective Clothing and Equipment',
     'rating': 4.8}},
   {'_index': 'shopee',
    '_type': '_doc',
    '_id': '2',
    '_score': 1.0,
    '_source': {'item_id': 2,
     'item_name': 'HUAWEI MateBook D15 (15.6" FHD IPS Laptop / AMD Ryzen 5 3500U + Radeon Vega 8 / 8GB RAM)',
     'item_price': 798.0,
     'category': 'Computers & Peripherals',
     'rating': 4.9}},
   {'_index': 'shopee',
    '_type': '_doc',
    '_id': '3',
    '_score': 1.0,
    '_source': {'item_id': 3,
     'item_name': 'Mohamm English Letter & Number Sticker',
     'item_price'